In [1]:
import pandas as pd
import requests
import json
import pickle as pkl
from tqdm.auto import tqdm
tqdm.pandas()


In [2]:
df = pd.read_csv('../data/data_with_altmetric.csv')
df = df[~df['doi'].isna()]
df = df[~df['abstract'].isna()]

/tmp/ipykernel_3439/2835532993.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/data_with_altmetric.csv')


In [3]:
df.shape

(14492, 43)

In [4]:
def get_openalex_attributes(doi: str):
    url = f'https://api.openalex.org/works/doi:{doi}'
    try:
        resp = requests.get(url, timeout=10)
        content = json.loads(resp.content)
    except:
        print('Error occured')
        return None

    result_dict = {}
    result_dict['type'] = content['type']
    result_dict['host_display_name'] = content['host_venue']['display_name']
    result_dict['publisher'] = content['host_venue']['publisher']
    result_dict['is_open_access'] = content['open_access']['is_oa']
    result_dict['open_alex_citations_count'] = content['cited_by_count']
    inst_nested_list = [el['institutions'] for el in content['authorships']]
    inst_nested_names = list(map(lambda inst_list: [entry['display_name'] for entry in inst_list], inst_nested_list))
    result_dict['institutions'] = [el for subl in inst_nested_names for el in subl]

    return result_dict

In [5]:
respones = df['doi'].progress_apply(get_openalex_attributes)

  0%|          | 0/14492 [00:00<?, ?it/s]

Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error 

In [6]:
with open('../data/openalex_respones.pkl', 'wb') as f:
    pkl.dump(respones, f)

In [8]:
respones.isna().sum()

121

In [14]:
df = df.loc[~respones.isna()].reset_index(drop=True)
respones = respones.loc[~respones.isna()]
rows = respones.apply(pd.DataFrame)
responses_df = pd.concat(rows.to_list(), axis='index').reset_index(drop=True)
df_with_openalex = pd.concat([df, responses_df], axis='columns')

In [19]:
df_with_openalex.to_csv('../data/df_all.csv', index=False)